#### Overview

위경도 정보가 포함된 CSV파일을 geometry의 하나인 Point 공간데이터로 변경

#### Import Libraries

In [2]:
import os
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# High-Resolution Graph Output option for Retina Displays
%config InlineBackend.figure_format = 'retina'

#### Fetching Data with CSV file
* 공공데이터포털에 있는 `전국도서관표준데이터`

In [ ]:
FILE_PATH = os.path.join(os.getcwd(), '89_data', 'standard-libraries.csv')
lib_df = pd.read_csv(FILE_PATH)
lib_df.info()

#### Extracting Specific District Names

In [ ]:
DISTRICT_NAME = '남양주시'

In [ ]:
lib_df_nam = lib_df[lib_df.시군구명==DISTRICT_NAME]
lib_df_nam.info()

In [ ]:
lib_df_nam.head()

In [ ]:
lib_df_nam['위도'].unique().size

#### (x,y):CSV to Geometry info

In [ ]:
geometry = gpd.points_from_xy(lib_df_nam.경도, lib_df_nam.위도)

In [ ]:
geometry

In [ ]:
lib_gdf_nam = gpd.GeoDataFrame(lib_df_nam, geometry=geometry, crs='epsg:4326')
lib_gdf_nam.info()
# 마지막에 `geometry` 컬럼 추가

In [ ]:
lib_gdf_small = lib_gdf_nam[['도서관명', '도서관유형', '열람좌석수', '건물면적', 'geometry']]
lib_gdf_small.head()

In [ ]:
lib_gdf_small.columns = ['name', 'type', 'chair_cnt', 'bd_area', 'geometry']
lib_gdf_small.head()

In [ ]:
lib_gdf_small.plot(figsize=(10,10))
plt.show()  

#### Fetching VW-LAB Data(GeoJSON)

In [ ]:
SHP_PATH = os.path.join(os.getcwd(), '89_data', 'vw-lab.HangJeongDong_ver20220701.geojson')
dong_gdf = gpd.read_file(SHP_PATH)  
dong_gdf.info()

* Displaing Multi-Layers

In [ ]:
namyangju_layer = dong_gdf[dong_gdf.sggnm==DISTRICT_NAME]\
    .plot(figsize=(10,10), color='lightgray', edgecolors='white')
lib_gdf_small.plot(ax=namyangju_layer)
plt.show()

#### Save SHP file

In [ ]:
NEW_SHP_PATH = os.path.join(
    os.getcwd(),
    '89_data',
    'new-libraries-geometry.shp'
)

lib_gdf_small.to_file(NEW_SHP_PATH, encoding='cp949')
# 파일이 5개 생성됨
# 'new-libraries-geometry.shp', 
# 'new-libraries-geometry.cpg',
# 'new-libraries-geometry.dbf',
# 'new-libraries-geometry.prj',
# 'new-libraries-geometry.shx'

#### Fetching Data with JSON file
* 공공데이터 포털의 `전국문화축제표준데이터` 

In [3]:
# Cultural-Festival.json
# 이 파일이 없으면 08_save_web_data.ipynb의 'JSON'부분 실행
CF_PATH = os.path.join(os.getcwd(), '89_data', 'cultural-festival.json')
with open(CF_PATH, 'r', encoding='utf-8') as cf_json:
    json_data = json.load(cf_json)

In [29]:
for key in json_data.keys():
    print(key)

fields
records


In [30]:
_sample_data = json_data.get('records', [])[:2]
_sample_data

[{'축제명': '춘천마임축제 봄시즌',
  '개최장소': '춘천시내 다수지역',
  '축제시작일자': '2021-05-23',
  '축제종료일자': '2021-05-29',
  '축제내용': '2021년 우리의 잃어버린 봄을 찾기위한 무언의 몸짓으로 시민과 예술가의 봄을 찾는 공연',
  '주관기관': '(사)춘천마임축제',
  '주최기관': '(사)춘천마임축제+KBS방송총국+춘천MBC',
  '후원기관': '문화체육관광부+강원도청+강원도 춘천시청+한국관광공사+남이섬교육문화그룹+나미나라공화국남이섬+한림대학교춘천성심병원+한국문화예술위원회+한국메세나협회',
  '전화번호': '033-242-0585',
  '홈페이지주소': 'https://www.cccf.or.kr/Home/H40000/H40400/boardView?board_key=7642&search_type=1&search_keyword=ECB698ECB29CEBA788EC9E84ECB695ECA09C+EBB484EC8B9CECA68C',
  '관련정보': '전시관람',
  '소재지도로명주소': '강원도 춘천시 춘천로 112 축제극장몸짓',
  '소재지지번주소': '강원도 춘천시 효자동 531',
  '위도': '37.87308900',
  '경도': '127.72784500',
  '데이터기준일자': '2023-01-12',
  '제공기관코드': '4181000',
  '제공기관명': '강원특별자치도 춘천시'},
 {'축제명': '춘천인형극제 코코바우 놀이터 자연아 놀러와',
  '개최장소': '춘천인형극장',
  '축제시작일자': '2021-04-24',
  '축제종료일자': '2021-07-24',
  '축제내용': '자연의 흐름을 주제로 모두 함께 인형극, 만들기를 즐기며 자연을 느낄 수 있다.',
  '주관기관': '춘천인형극제',
  '주최기관': '춘천인형극제',
  '후원기관': '강원도 춘천시청',
  '전화번호': '033-242-8452',
  '홈페이지주소': 'https://www.

* JSON to DataFrame

In [4]:
cf_df = pd.json_normalize(json_data['records'])
cf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1707 entries, 0 to 1706
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   축제명       1707 non-null   object
 1   개최장소      1707 non-null   object
 2   축제시작일자    1707 non-null   object
 3   축제종료일자    1707 non-null   object
 4   축제내용      1707 non-null   object
 5   주관기관      1707 non-null   object
 6   주최기관      1707 non-null   object
 7   후원기관      1707 non-null   object
 8   전화번호      1707 non-null   object
 9   홈페이지주소    1707 non-null   object
 10  관련정보      1707 non-null   object
 11  소재지도로명주소  1707 non-null   object
 12  소재지지번주소   1707 non-null   object
 13  위도        1707 non-null   object
 14  경도        1707 non-null   object
 15  데이터기준일자   1707 non-null   object
 16  제공기관코드    1707 non-null   object
 17  제공기관명     1707 non-null   object
dtypes: object(18)
memory usage: 240.2+ KB


In [32]:
cf_df['경도'].value_counts().sort_values(ascending=False)[:5]
# '경도'값이 공백인 row가 217개 존재함(공백은 숫자로 변환이 안되므로 제거)

경도
                217
127.73137363     34
128.5965647      29
127.72934048     28
129.1635303      17
Name: count, dtype: int64

In [5]:
# '경도'값이 공백인 row삭제
cf_df.drop(cf_df[cf_df.경도 == ''].index, inplace=True)

In [34]:
# 다시 확인
cf_df['경도'].value_counts().sort_values(ascending=False)[:5]

경도
127.73137363    34
128.5965647     29
127.72934048    28
129.1635303     17
129.0082172     16
Name: count, dtype: int64

In [6]:
geometry_cf = gpd.points_from_xy(cf_df.경도, cf_df.위도)

In [7]:
cf_gdf = gpd.GeoDataFrame(cf_df, geometry=geometry_cf, crs='epsg:4326')
cf_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 1490 entries, 0 to 1706
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   축제명       1490 non-null   object  
 1   개최장소      1490 non-null   object  
 2   축제시작일자    1490 non-null   object  
 3   축제종료일자    1490 non-null   object  
 4   축제내용      1490 non-null   object  
 5   주관기관      1490 non-null   object  
 6   주최기관      1490 non-null   object  
 7   후원기관      1490 non-null   object  
 8   전화번호      1490 non-null   object  
 9   홈페이지주소    1490 non-null   object  
 10  관련정보      1490 non-null   object  
 11  소재지도로명주소  1490 non-null   object  
 12  소재지지번주소   1490 non-null   object  
 13  위도        1490 non-null   object  
 14  경도        1490 non-null   object  
 15  데이터기준일자   1490 non-null   object  
 16  제공기관코드    1490 non-null   object  
 17  제공기관명     1490 non-null   object  
 18  geometry  1490 non-null   geometry
dtypes: geometry(1), object(18)
memory usage: 232.

In [8]:
mini_cf_gdf = cf_gdf[['축제명', '제공기관명', 'geometry']]
mini_cf_gdf.head()

,축제명,제공기관명,geometry
0,춘천마임축제 봄시즌,강원특별자치도 춘천시,POINT (127.72785 37.87309)
1,춘천인형극제 코코바우 놀이터 자연아 놀러와,강원특별자치도 춘천시,POINT (127.72020 37.92102)
2,춘천마임축제 걷다 보는 마임,강원특별자치도 춘천시,POINT (127.72785 37.87309)
3,춘천마임축제 대학로 문화의 거리,강원특별자치도 춘천시,POINT (127.72785 37.87309)
4,제33회 춘천인형극제 국내 참가작 공모,강원특별자치도 춘천시,POINT (127.72020 37.92102)


In [12]:
# ESRI Shapefile에서 컬럼명이 10자를 초과하면 잘리기 때문에
# `organization`을 `org`로 변경
mini_cf_gdf.columns = ['cf_name', 'org', 'geometry']
mini_cf_gdf.head()

,cf_name,org,geometry
0,춘천마임축제 봄시즌,강원특별자치도 춘천시,POINT (127.72785 37.87309)
1,춘천인형극제 코코바우 놀이터 자연아 놀러와,강원특별자치도 춘천시,POINT (127.72020 37.92102)
2,춘천마임축제 걷다 보는 마임,강원특별자치도 춘천시,POINT (127.72785 37.87309)
3,춘천마임축제 대학로 문화의 거리,강원특별자치도 춘천시,POINT (127.72785 37.87309)
4,제33회 춘천인형극제 국내 참가작 공모,강원특별자치도 춘천시,POINT (127.72020 37.92102)


In [41]:
# gdf.explore()가 실행되기 위해서는
# 'folium', 'mapclassify' 라이브러리가 가상환경에 설치되어야 함(ipynb에 임포트할 필요는 없음)
mini_cf_gdf.explore('organization', categorical=True)

* Save SHP file

In [13]:
NEW_CF_SHP_PATH = os.path.join(
    os.getcwd(),
    '89_data',
    'mini_cultural-festival.shp'
)

mini_cf_gdf.to_file(NEW_CF_SHP_PATH, encoding='cp949')